In [1]:
from Sindonews import Sindonews #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [5]:
obj = Sindonews()
end_date = date(2018, 7, 24)
start_date = date(2018, 5, 12)
init_page = 3
init_offset = 20

In [7]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, init_offset, single_date.strftime("%Y-%m-%d"))
    init_page = 1
    init_offset = 0

2018-05-12
page  1
https://international.sindonews.com/read/1305234/43/palestina-kesal-sekutu-as-blokir-ue-kecam-relokasi-kedutaan-1526144353
Insert berita  Palestina Kesal Sekutu AS Blokir UE Kecam Relokasi Kedutaan
masuk
https://sports.sindonews.com/read/1305223/50/sengaja-tarung-lawan-dua-petinju-kidal-ini-alasan-linares-1526137346
Insert berita  Sengaja Tarung Lawan Dua Petinju Kidal, Ini Alasan Linares
masuk
https://sports.sindonews.com/read/1305233/51/menpora-resmi-melaunching-sepeda-nusantara-2018-1526142952
Insert berita  Menpora Resmi Melaunching Sepeda Nusantara 2018
masuk
https://soccer.sindonews.com/read/1305232/58/lumat-persija-madura-united-nodai-sugbk-1526142945
Insert berita  Lumat Persija, Madura United Nodai SUGBK
masuk
https://international.sindonews.com/read/1305231/40/selasa-depan-anwar-ibrahim-hirup-udara-bebas-1526142651
Insert berita  Selasa Depan, Anwar Ibrahim Hirup Udara Bebas
masuk
https://soccer.sindonews.com/read/1305228/59/pelatih-dc-united-akui-kepingin-

KeyboardInterrupt: 